In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ipywidgets as widgets
from IPython.display import display

In [2]:
base_path = "results/"
folders= ["share", "no_share"]
tests= ["multiplexer", "parity", "regression", "old_regression"]
# tests= ["multiplexer", "parity", "regression"]
runs = ["1", "2", "3", "4", "5"]
col_names = []

dfs = {}
for t in tests: 
    dfs[t] = {}
    for f in folders:
        dfs[t][f] = []
        for r in runs:
#             file_name = base_path + f + "/diversity_" + t + "_"+ r + "_n1000.csv"
            file_name = base_path + f + "/diversity_" + t + "_" + r  + ".csv"
            df = pd.read_csv(file_name, sep=';')
            dfs[t][f].append(df)
            col_names = df.columns[1:]
            
dfs[t][f][0].head(2)

,gen,fitness,n_species,p_isomorphisms,unique_fitness,entropy,total_gen_markers,active_gen_markers,inactive_gen_markers,avg_delta,std_delta
0,0,93.238610,9,282,24,0.409743,39219,3893,37508,0.792692,0.022616
1,1,78.657057,11,285,23,0.476600,38812,3663,37301,0.785470,0.031752


In [3]:
def normalize_column(df, c_name):
    c = df[c_name]
    df[c_name] = (c-c.min())/(c.max()-c.min())

In [4]:
def plot_graphs(dfs, test_name, gen_range, cols):
    for k, v in dfs[test_name].items():
#     if k == "share":
#         continue
        for column in cols:
            full_data = [d[column] for d in v]
            data = []
            for d in zip(*full_data):
                data.append(d)
            data = data[gen_range[0]:gen_range[1]]
            avg_arr = [sum(d)/len(d) for d in data]
            min_arr = [min(d) for d in data]
            max_arr = [max(d) for d in data]
            steps = [i for i in range(gen_range[0], gen_range[1])]
            plt.plot(steps, avg_arr, lw =1.3, label=f"{k} {column}")
            plt.fill_between(steps, min_arr,max_arr, alpha = 0.15)

    plt.rcParams['figure.figsize'] = [14, 8]
    plt.legend(loc="upper left")
    plt.show(block=False)

In [9]:
test = widgets.ToggleButtons(
    options=tests,
    description='Test:',
    disabled=False,
    button_style='',
)

gen_range = widgets.IntRangeSlider(
    value=[10, 999],
    min=0,
    max=999,
    step=10,
    description='Generations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

parameters = []
for col in col_names:
    cb = widgets.Checkbox(
        value=False,
        description=col,
        disabled=False,
        indent=False
    )
    parameters.append(cb)

    
    
display(test)
display(gen_range)

half_len = len(parameters)//2
input_widgets_1 = widgets.HBox(parameters[:half_len])
display(input_widgets_1)
input_widgets_2 = widgets.HBox(parameters[half_len:])
display(input_widgets_2)

button = widgets.Button(description="Render Graph")
output = widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        cols = [p.description for p in parameters if p.value]
        if len(cols) > 0:
            plot_graphs(dfs, test.value, gen_range.value, cols)

button.on_click(on_button_clicked)


ToggleButtons(description='Test:', options=('multiplexer', 'parity', 'regression', 'old_regression'), value='m…

IntRangeSlider(value=(10, 999), continuous_update=False, description='Generations:', max=999, step=10)

Button(description='Render Graph', style=ButtonStyle())

Output()